Final Project
* Rucha Athavale, Rishikesh Mane, Parita Shah, Shaildeep Kaur
- Deep within the Earth it is so hot that some rocks slowly melt and become a thick flowing substance called magma. Since it is lighter than the solid rock around it, magma rises and collects in magma chambers. Eventually, some of the magma pushes through vents and fissures to the Earth&#39;s surface. Magma that has erupted is called lava. Explosive volcanic eruptions can be dangerous and deadly. They can blast out clouds of hot tephra from the side or top of a volcano. These fiery clouds race down mountainsides destroying almost everything in their path. Ash erupted into the sky falls back to Earth like powdery snow. If thick enough, blankets of ash can suffocate plants, animals, and humans. When hot volcanic materials mix with water from streams or melted snow and ice, mudflows form. Mudflows have buried entire communities located near erupting volcanoes. How do scientist study volcanoes and predict eruptions early? Scientists use a wide variety of techniques to monitor volcanoes, including seismographic detection of the earthquakes and tremor that almost always precede eruptions, precise measurements of ground deformation that often accompanies the rise of magma, changes in volcanic gas emissions, and changes in gravity and magnetic fields. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        continue

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
from sklearn import metrics as metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error as mse
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
from sklearn.cluster import KMeans, MeanShift, DBSCAN
from sklearn.metrics import silhouette_score

from xgboost import XGBRegressor
import xgboost as xgb
import math

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import glob
import math

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

import optuna
from optuna.samplers import TPESampler

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
train = pd.read_csv("../input/predict-volcanic-eruptions-ingv-oe/train.csv")
sample_submission = pd.read_csv("../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv")

In [ ]:
fig = px.histogram(
    train, 
    x="time_to_eruption",
    width=800,
    height=500,
    nbins=100,
    title='Time to erupt distribution'
)

fig.show()


In [ ]:
fig = px.line(
    train, 
    y="time_to_eruption",
    width=800,
    height=500,
    title='Time to erupt for all volcanos'
)

fig.show()

In [ ]:
train['time_to_eruption'].describe()

In [ ]:
print('Median:', train['time_to_eruption'].median())
print('Skew:', train['time_to_eruption'].skew())
print('Std:', train['time_to_eruption'].std())
print('Kurtosis:', train['time_to_eruption'].kurtosis())
print('Mean:', train['time_to_eruption'].mean())

In [ ]:
train_sensor = glob.glob("../input/predict-volcanic-eruptions-ingv-oe/train/*")
len(train_sensor)

In [ ]:
test_sensor = glob.glob("../input/predict-volcanic-eruptions-ingv-oe/test/*")
len(test_sensor)

In [ ]:
example = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train/2037160701.csv')
example.head()

In [ ]:
sensors = set()
observations = set()
nan_columns = list()
missed_groups = list()
for_df = list()

for item in train_sensor:
    name = int(item.split('.')[-2].split('/')[-1])
    at_least_one_missed = 0
    frag = pd.read_csv(item)
    missed_group = list()
    missed_percents = list()
    for col in frag.columns:
        missed_percents.append(frag[col].isnull().sum() / len(frag))
        if pd.isnull(frag[col]).all() == True:
            at_least_one_missed = 1
            nan_columns.append(col)
            missed_group.append(col)
    if len(missed_group) > 0:
        missed_groups.append(missed_group)
    sensors.add(len(frag.columns))
    observations.add(len(frag))
    for_df.append([name, at_least_one_missed] + missed_percents)

In [ ]:
print('Number of sensors: ', sensors)
print('Number of observations: ', observations)

In [ ]:
print('Number of missed sensors:', len(nan_columns))

absent_sensors = dict()

for item in nan_columns:
    if item in absent_sensors:
        absent_sensors[item] += 1
    else:
        absent_sensors[item] = 0

In [ ]:
absent_df = pd.DataFrame(absent_sensors.items(), columns=['Sensor', 'Missed sensors'])

fig = px.bar(
    absent_df, 
    x="Sensor",
    y='Missed sensors',
    width=800,
    height=500,
    title='Number of missed sensors in training dataset'
)

fig.show()

In [ ]:
for_df = pd.DataFrame(
    for_df, 
    columns=[
        'segment_id', 'has_missed_sensors', 'missed_%_sensor1', 
        'missed_%_sensor2', 'missed_%_sensor3', 'missed_%_sensor4', 
        'missed_%_sensor5', 'missed_%_sensor6', 'missed_%_sensor7', 
        'missed_%_sensor8', 'missed_%_sensor9', 'missed_%_sensor10'
    ]
)
for_df

In [ ]:
train = pd.merge(train, for_df)
train

In [ ]:
sensors = set()
observations = set()
nan_cols = list()
for_test_df = list()

for item in test_sensor:
    name = int(item.split('.')[-2].split('/')[-1])
    at_least_one_missed = 0
    sensor = pd.read_csv(item)
    missed_percents = list()
    for col in sensor.columns:
        missed_percents.append(sensor[col].isnull().sum() / len(sensor))
        if pd.isnull(sensor[col]).all() == True:
            at_least_one_missed = 1
            nan_cols.append(col)
    sensors.add(len(sensor.columns))
    observations.add(len(sensor))
    for_test_df.append([name, at_least_one_missed] + missed_percents)

In [ ]:
for_test_df = pd.DataFrame(
    for_test_df, 
    columns=[
        'segment_id', 'has_missed_sensors', 'missed_%_sensor1', 'missed_%_sensor2', 'missed_%_sensor3', 
        'missed_%_sensor4', 'missed_%_sensor5', 'missed_%_sensor6', 'missed_%_sensor7', 
        'missed_%_sensor8', 'missed_%_sensor9', 'missed_%_sensor10'
    ]
)

for_test_df

In [ ]:
print('Number of sensors: ', sensors)
print('Number of observations: ', observations)

In [ ]:
print('Number of missed sensors:', len(nan_columns))

absent_sensors = dict()

for item in nan_cols:
    if item in absent_sensors:
        absent_sensors[item] += 1
    else:
        absent_sensors[item] = 0

In [ ]:
absent_df = pd.DataFrame(absent_sensors.items(), columns=['Sensor', 'Missed sensors'])

fig = px.bar(
    absent_df, 
    x="Sensor",
    y='Missed sensors',
    width=800,
    height=500,
    title='Number of missed sensors in testing dataset'
)

fig.show()

In [ ]:
fig = make_subplots(rows=5, cols=2)
traces = [
    go.Histogram(
        x=example[col], 
        nbinsx=100, 
        name=col
    ) for col in example.columns
]

for i in range(len(traces)):
    fig.append_trace(
        traces[i], 
        (i // 2) + 1, 
        (i % 2) + 1
    )

fig.update_layout(
    title_text='Data from sensors distribution',
    height=800,
    width=1200
)

fig.show()

In [ ]:
fig = make_subplots(rows=5, cols=2)
traces = [
    go.Scatter(
        x=[i for i in range(60002)], 
        y=example[col], 
        mode='lines', 
        name=col
    ) for col in example.columns
]

for i in range(len(traces)):
    fig.append_trace(
        traces[i], 
        (i // 2) + 1, 
        (i % 2) + 1
    )

fig.update_layout(
    title_text='Data from sensors',
    height=800,
    width=1200
)

fig.show()

In [ ]:
def features(feats, ts, sensor_id):
    X = pd.DataFrame()
    f = np.fft.fft(feats)
    f_real = np.real(f)
    X.loc[ts, f'{sensor_id}_sum']       = feats.sum()
    X.loc[ts, f'{sensor_id}_mean']      = feats.mean()
    X.loc[ts, f'{sensor_id}_std']       = feats.std()
    X.loc[ts, f'{sensor_id}_var']       = feats.var() 
    X.loc[ts, f'{sensor_id}_max']       = feats.max()
    X.loc[ts, f'{sensor_id}_min']       = feats.min()
    X.loc[ts, f'{sensor_id}_skew']      = feats.skew()
    X.loc[ts, f'{sensor_id}_mad']       = feats.mad()
    X.loc[ts, f'{sensor_id}_kurtosis']  = feats.kurtosis()
    return X

In [ ]:
train_set = list()
j = 0
for seg in train.segment_id:
    files = pd.read_csv(f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/train/{seg}.csv')
    train_row = []
    if j%500 == 0:
        print(j)
    for i in range(0, 10):
        sensor_id = f'sensor_{i+1}'
        train_row.append(features(files[sensor_id].fillna(0), seg, sensor_id))
    train_row = pd.concat(train_row, axis=1)
    train_set.append(train_row)
    j+=1

train_set = pd.concat(train_set)

In [ ]:
train_set = train_set.reset_index()
train_set = train_set.rename(columns={'index': 'segment_id'})
train_set = pd.merge(train_set, train, on='segment_id')
train_set

In [ ]:
test_set = list()
j=0
for seg in sample_submission.segment_id:
    files = pd.read_csv(f'/kaggle/input/predict-volcanic-eruptions-ingv-oe/test/{seg}.csv')
    test_row = []
    if j%500 == 0:
        print(j)
    for i in range(0, 10):
        sensor_id = f'sensor_{i+1}'
        test_row.append(features(files[sensor_id].fillna(0), seg, sensor_id))
    test_row = pd.concat(test_row, axis=1)
    test_set.append(test_row)
    j+=1
test_set = pd.concat(test_set)

In [ ]:
test_set = test_set.reset_index()
test_set = test_set.rename(columns={'index': 'segment_id'})
test_set = pd.merge(test_set, for_test_df, on='segment_id')
test = test_set.drop(['segment_id'], axis=1)
test

In [ ]:
drop_cols = list()
for col in train_set.columns:
    if col == 'segment_id':
        continue
    if abs(train_set[col].corr(train_set['time_to_eruption'])) < 0.01:
        drop_cols.append(col)
drop_cols

In [ ]:
no_drop_cols = list()

for col1 in train_set.columns:
    for col2 in train_set.columns:
        if col1 == col2:
            continue
        if col1 == 'segment_id' or col2 == 'segment_id': 
            continue
        if col1 == 'time_to_eruption' or col2 == 'time_to_eruption':
            continue
        if abs(train_set[col1].corr(train_set[col2])) > 0.98:
            if col2 not in drop_cols and col1 not in no_drop_cols:
                drop_cols.append(col2)
                no_drop_cols.append(col1)
no_drop_cols

* Variance check is inlcuded in the features: '{sensor_id}_variance'_
* Correlation check is included in dropping columns

In [ ]:
train_set.head()

In [ ]:
test_set.head()

Logistic Regression

In [ ]:
X = train_set.drop(['time_to_eruption'], axis = 1)
y = train_set['time_to_eruption']
X_test = test_set
#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

#logmodel = LogisticRegression()
#logmodel.fit(X, y)
#pred_1 = logmodel.predict(X_test)
#pred_1

#print(classification_report(y_test, pred_1))

* print('Mean Absolute Error:', metrics.mean_absolute_error(y, pred_1))  
* print('Mean Squared Error:', metrics.mean_squared_error(y, pred_1))  
* print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, pred_1)))
* print('R-squared Error:', metrics.r2_score(y, pred_1))

In [ ]:
#X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.9)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)
pred_2 = knn.predict(X_test)
pred_2

* print('Mean Absolute Error:', metrics.mean_absolute_error(y, pred_2))  
* print('Mean Squared Error:', metrics.mean_squared_error(y, pred_2))  
* print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, pred_2)))
* print('R-squared Error:', metrics.r2_score(y, pred_2))

Linear Regression

In [ ]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X)
X_test_sc = sc.transform(X_test)
regressor = LinearRegression()  
regressor.fit(X, y)
print(regressor.intercept_)
print(regressor.coef_)
pred_3 = regressor.predict(X_test)
pred_3

plt.scatter(X_test, y_test,  color='gray')
plt.plot(X_test, y_pred, color='red', linewidth=2)
plt.show()

* print('Mean Absolute Error:', metrics.mean_absolute_error(y, pred_3))  
* print('Mean Squared Error:', metrics.mean_squared_error(y, pred_3))  
* print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y, pred_3)))
* print('R-squared Error:', metrics.r2_score(y, pred_3))

K-Means Cluster

def kmeans(model):
    pipeline = Pipeline([
        ('kmeans', KMeans()),
        (f'{model}', model)
    ])
    pipeline.fit(X, y)
    preds = pipeline.predict(X_test)
    return(mean_squared_error(y, preds))

Running K-means with K = 3

In [ ]:
kmeans = KMeans(n_clusters = 3, max_iter = 50, random_state = 14)
kmeans.fit(X)
kmeans.labels_ 
pred_4 = kmeans.predict(X) 
pred_4

Density based Clustering

In [ ]:
def DBSCAN_params(max_distance,min_cars,ride_data):
    
    ## get coordinates from ride data
    coords = ride_data.as_matrix(columns=['lat', 'lon'])
    
    ## calculate epsilon parameter using
    ## the user defined distance
    kms_per_radian = 6371.0088
    epsilon = max_distance / kms_per_radian
    
    ## perform clustering
    db = DBSCAN(eps=epsilon, min_samples=min_cars,
                algorithm='', metric='').fit(np.radians(coords))
    
    ## group the clusters
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])
    
    ## report
    print('Number of clusters: {}'.format(num_clusters))

    ## initialize lists for hot spots
    lat = []
    lon = []
    num_members = []
 
    
    ## loop through clusters and get centroids, number of members
    for ii in range(len(clusters)):
 
        ## filter empty clusters
        if clusters[ii].any():
 
            ## get centroid and magnitude of cluster
            lat.append(MultiPoint(clusters[ii]).centroid.x)
            lon.append(MultiPoint(clusters[ii]).centroid.y)
            num_members.append(len(clusters[ii]))
            
    params = [lon,lat,num_members]
    return params 
  

In [ ]:
from sklearn.cluster import DBSCAN
from itertools import product

eps_values = np.arange(8,12.75,0.25) # eps values to be investigated
min_samples = np.arange(3,10) # min_samples values to be investigated
DBSCAN_params = list(product(eps_values, min_samples))
no_of_clusters = []
sil_score = []

for p in DBSCAN_params:
    DBS_clustering = DBSCAN(eps=p[0], min_samples=p[1]).fit(X)
    no_of_clusters.append(len(np.unique(DBS_clustering.labels_)))
    #sil_score.append(silhouette_score(X, DBS_clustering.labels_))
    #print(no_of_clusters, silhouette_score(X,DBS_clustering))
    #print(DBS_clustering)
    dbs = np.zeros_like(DBS_clustering.labels_, dtype=bool)
    dbs[DBS_clustering.core_sample_indices_] = True
    labels = DBS_clustering.labels_

In [ ]:
#plt.scatter(X[:,0],s=10,c=labels)

In [ ]:
#tmp = pd.DataFrame.from_records(DBSCAN_params, columns =['Eps', 'Min_samples'])   
#tmp['Sil_score'] = sil_score

#pivot_1 = pd.pivot_table(tmp, values='Sil_score', index='Min_samples', columns='Eps')

#fig, ax = plt.subplots(figsize=(18,6))
#sns.heatmap(pivot_1, annot=True, annot_kws={"size": 10}, cmap="YlGnBu", ax=ax)
#plt.show()
DBS_clustering = DBSCAN(eps=12.5, min_samples=4).fit(X)

DBSCAN_clustered = X.copy()
DBSCAN_clustered.loc[:,'Cluster'] = DBS_clustering.labels_ # append labels to points
DBSCAN_clust_sizes = DBSCAN_clustered.groupby('Cluster').size().to_frame()
DBSCAN_clust_sizes.columns = ["DBSCAN_size"]
DBSCAN_clust_sizes

XGBoost

In [ ]:
eval_metric = ["rmse"]
xgb_model = XGBRegressor()
XGBRegressor(base_score=0.5, booster='gbtree',
       importance_type='gain', learning_rate=0.3, max_delta_step=0,
       max_depth=3, min_child_weight=1, n_estimators=100,
       n_jobs=1,objective='reg:linear', random_state=0,
        seed=None,silent=None, subsample=1, verbosity=1)

xgb_model.fit(X, y, eval_metric = eval_metric)

from sklearn.metrics import precision_recall_fscore_support as score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score
pred_5 = xgb_model.predict(X_test)
pred_5

In [ ]:
pred_5.shape

* from sklearn.metrics import mean_absolute_error, mean_squared_error
* from math import sqrt
* print("MAE test score:", mean_absolute_error(y, pred_5))
* print("MSE test score:", mean_squared_error(y, pred_5))
* print("RMSE test score:", sqrt(mean_squared_error(y, pred_5)))

In [ ]:
#x_ax = range(len(y))
#plt.plot(x_ax, y, label="original")
#plt.plot(x_ax, pred_5, label="predicted")
#plt.title("test and predicted data")
#plt.legend()
#plt.show()

In [ ]:
test_set['time_to_eruption'] = pred_5
#test_set['time_to_eruption'] = pred_3
#test_set['time_to_eruption'] = pred_2
#test_set['time_to_eruption'] = pred_1

In [ ]:
sample_submission = pd.merge(sample_submission, test_set[['segment_id', 'time_to_eruption']], on='segment_id')

In [ ]:
sample_submission = sample_submission.drop(['time_to_eruption_x'], axis=1)
sample_submission.columns = ['segment_id', 'time_to_eruption']
sample_submission

In [ ]:
sample_submission.to_csv('submission.csv', index=False)

References
* https://www.kaggle.com/isaienkov

Overall, our models are working good but we got some errors during the classification report as there was some index mismatch. We have learned a lot of things from this project, it taught how important data cleaning, data visualization, and data imputation is because you cannot drop the columns with missing values as we are losing a lot of valuable data, instead we filled it with either median or mode. Data pre-processing was time-consuming because there was a massive amount of data we had to work on but we had a good learning experience working with large datasets.